<a href="https://colab.research.google.com/github/Arseniy-Polyakov/applied_linguistics_course/blob/main/Task_1_Corpus_Linguistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Данная работа посвящена генерации открыток (дореволюционные открытки на тему путешествия) в стиле корпуса "Пишу тебе" с помощью моделей семейства GigaChat (GigaChat, GigaChat2, GigaChat-Max, GigaChat-Pro, GigaChat-2-Max, GigaChat-2-Pro).
Дореволюционные открытки являются достаточно сложным атрибутом для генерации с помощью LLM, так как они написаны в более свободном стиле, с метафоричностью высказываний, без формализма и шаблонности (которые присущи открытками советского периода). Данные особенности усложняют процесс генерации, в связи с этим будет проведен эксперимент по определению наиболее удачной модели для данной задачи.

Устанавливаем необходимые библиотеки для работы с GigaChat API и пост-обработки данных

In [111]:
!pip install gigachat json_repair

In [180]:
import re
import requests
import json_repair
import pandas as pd
from google.colab import userdata
from gigachat import GigaChat

Инициализируем переменную с токеном авторизации в Studio Sber

In [30]:
GIGA_API_KEY = userdata.get("GIGA_API_KEY")

Пишем REST API функцию для получения токена доступа (access_token) для обращения к LLM

In [80]:
def get_access_token(AUTHORIZATION_KEY: str, RQUID: str) -> str:
  url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
  headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json",
    "RqUID": RQUID,
    "Authorization": "Basic " + GIGA_API_KEY
  }

  payload={
    "scope": "GIGACHAT_API_PERS"
  }

  response = requests.request("POST", url, headers=headers, data=payload, verify=False)

  response_json = response.json()
  access_token = response_json["access_token"]
  return access_token

In [154]:
access_token = get_access_token(GIGA_API_KEY, "51dd89ba-4b59-479b-8437-b8f0ee4a9135")

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Выводим список доступных моделей

In [47]:
def get_avaliable_models(ACCESS_TOKEN: str):
  url = "https://gigachat.devices.sberbank.ru/api/v1/models"
  headers = {
      "Authorization": "Bearer " + ACCESS_TOKEN
  }
  response = requests.get(url, headers=headers, verify=False)
  return response.json()

In [95]:
models = get_avaliable_models(access_token)
model = models["data"]
model

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'id': 'GigaChat',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-2',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-2-Max',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-2-Max-preview',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-2-Pro',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-2-Pro-preview',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-2-preview',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-Max',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-Max-preview',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat-Plus',
  'object': 'model',
  'owned_by': 'salutedevices',
  'type': 'chat'},
 {'id': 'GigaChat

В ходе экcперимента мы будем использовать как и более ранние, так и SOTA модели семейства GigaChat (GigaChat, GigaChat2, GigaChat-Max, GigaChat-Pro, GigaChat-2-Max, GigaChat-2-Pro). В выборку моделей не войдут платные модели (GigaChat-Plus, его превью версия и вторая), а также все preview версии других моделей.

Напишем функцию обращения к модели

In [177]:
def chat_completion(model: str, ACCESS_TOKEN: str) -> str:
  url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
  headers = {
      "Authorization": "Bearer " + ACCESS_TOKEN
  }
  body = {
      "model": model,
      "messages": [{"role": "system", "content": """Ты человек, который пишет открытки в дереволюционную эпоху (до 1917 г.) на тему путешествий.
                                                    Напиши 10 открыток на русском языке.
                                                    Предерживайся следующей структуры открытки: приветствие -> рассказ о месте -> пожелание всего доброго -> прощание
                                                    Пиши открытки от первого лица.
                                                    Пиши открытки из тех мест, которые были популярны среди русских путешественников в 19 веке
                                                    (например: Константинополь, Крым, Баку, Рязань, Кисловодск, Москва)
                                                    Используй сокращения при написании открыток.
                                                    Используй уменьшительно-ласкательные обращения к близким, художественно-выразительные средства описания места, где ты находишься.
                                                    Ответ строго представь в формате JSON, где ключи JSON структуры это номера открыток, а значения их тексты.
                                                    В ответе строго предоставь только JSON структуру, не добаляй комментарии, не используй лишней информации.
                                                    Не пиши те же самые открытки, которые упоминаются в инструкции, используй их для вдохновения.
                                                    """},
                  {"role": "user", "content":   """Используй следующие открытки, чтобы написать свои в таком же стиле:
                                                     Шлю привет из Брянска, куда приехал в 11 ч[асов] дня.
                                                     По пути [?стоять] 3 часа. Так что время девать некуда.
                                                     В Киеве будем в 6 ч[асов] утра. Ехать хорошо.
                                                     Погода прекрасная. Целую крепко всех. [?Люблю].

                                                     Дорогая Анюта! Наконец мы у цели своего назначения. т.е. сейчас приехали в Новороссийск.
                                                     Местность замечательно красивая. Кругом Кавказские горы. Проехали большой туннель.
                                                     Сейчас идем осматривать городок и в 6 час[ов] веч[ера] уедем на пароходе по Черному морю в Анапу, где и будем иметь пребывание.
                                                     Сегодня уже ели дивные персики.
                                                     По приезде в Анапу напишем. Напиши или {если} будешь лично, кланяйся мамочке и всем нашим,
                                                     сейчас вспомнила, что ты теперь в Ловати {и пишу туда. Крепко целуем Маруся и Коля}

                                                     Привет Вам с Кавказа все горы кланяются низко.
                                                     Знаете ли вы о существовании нового источника, который по своему составу подходит к Смирновскому,
                                                     но гораздо горячее в нем 56° тепла.
                                                     {Я ходила кругом Желтой горы а завтра иду на ее вершину.
                                                     Теперь я понимаю почему вам было жаль уезжать отсюда такая красота}"""}],
      "temperature": 1
  }
  response = requests.post(url, headers=headers, json=body, verify=False)
  giga_answer = response.json()
  result = giga_answer["choices"][0]["message"]["content"]
  result_list = list(json_repair.loads(result).values())
  return result_list

Выведем полученный результат (сгенерированные открытки)

In [ ]:
result = chat_completion("GigaChat", access_token)
result

Выведем все полученные открытки в ходе эксперимента

In [183]:
texts_before_revolution_gigachat_pro = ['Приветствую тебя из Казани, милая моя сестрица! Великолепный город, раскинулся вдоль Волги величавой. Здесь так много интересного - мечети, церкви, музеи... А воздух какой чистый, просто чудо! Сегодня планирую посетить Кремль казанский, а вечером пойду на набережную, любоваться закатом. Передай всем родным мой сердечный привет. Жду с нетерпением встречи с тобой.',
 'Дорогая Софья, шлю тебе привет из Нижнего Новгорода! Ах, какая здесь красота! Кремль древний, улицы старинные, и люди добродушные. Виды на Волгу просто завораживают. Сегодня собираюсь подняться на башню кремлевскую, чтобы увидеть весь город с высоты птичьего полета. Как твои дела, дорогая? Надеюсь, скоро увидимся!',
 'Здравствуй, дорогой брат Владимир! Вот я и в Перми, городе на берегу могучей Камы. Здесь такие леса вокруг, что дух захватывает! Сегодня прогулялся до Егошихинского оврага, место историческое, связанное с основанием города. А вечером отправляюсь на экскурсию по городу. Береги себя, жду новостей от тебя!',
 'Приветствую тебя из Саратова, любимая моя жена! Город стоит на высоком берегу Волги, вид отсюда просто потрясающий. Сегодня побывал на Соколовой горе, там такой простор, что кажется, можно охватить взглядом всю Россию. Вечером планирую прогулку по набережной. Очень скучаю по тебе, жди меня скорее домой!',
 'Дорогая Мария, здравствуй! Я в Екатеринбурге, городе на границе Европы и Азии. Здесь столько интересных мест - памятники, музеи, театры... Сегодня посетил Храм-на-Крови, очень впечатляет своей историей и архитектурой. Вечером пойду гулять по набережной Исети. Как твои дела, радость моя? Жду нашей скорой встречи!',
 'Приветствую тебя из Иркутска, мой верный друг! Город расположен на берегах Ангары, окружен горами и тайгой. Сегодня был в музее декабристов, узнал много нового об истории нашего края. Вечером отправился на Байкал, наслаждаться его величием и красотой. Жду наших встреч, дружок!',
 'Здравствуй, дорогая Ольга! Вот я и в Архангельске, городе на Белом море. Здесь такие суровые красоты - северная природа, холодное море... Сегодня побывал в музее деревянного зодчества "Малые Карелы", настоящее чудо архитектуры под открытым небом. Вечером прогулялся по набережной Северной Двины. Очень скучаю по тебе, жду нашей встречи!',
 'Приветствую тебя из Красноярска, моя любимая дочь! Город стоит на берегах Енисея, окружен Саянскими горами. Сегодня посетил заповедник "Столбы" - удивительное место с уникальными скалами. Вечером пошел на смотровую площадку, откуда открывается вид на весь город. Как твои дела, солнышко мое? Жду нашей скорой встречи!',
 'Дорогая Лиза, здравствуй! Я в Чите, городе на востоке страны. Здесь такие просторы, тайга и степи сливаются воедино. Сегодня побывал в парке Одора, полюбовался природой Забайкалья. Вечером прогулялся по улице Ленина, сердце города. Как твои дела, радость моя? Жду нашей скорой встречи!',
 'Приветствую тебя из Владивостока, мой верный товарищ! Город на берегу Тихого океана, окружен сопками и бухтами. Сегодня побывал на Токаревской кошке, насладился видом на залив Золотой Рог. Вечером прогулялся по Светланской улице, главной артерии города. Как твои дела, дружище? Жду нашей скорой встречи!']

texts_before_revolution_gigachat_max = ['Приветствую тебя, мой дорогой друг, из живописного Крыма! Я прибыл сюда вчера вечером и сразу же отправился исследовать местные красоты. Крым — удивительное место, полное зелени и свежего воздуха. Вчера мы посетили знаменитые пещеры Чатыр-Дага, которые поражают своими размерами и красотой. Завтра планируем отправиться на прогулку вдоль побережья, чтобы насладиться видом моря и отдохнуть на пляже. Надеюсь, тебе тоже удастся когда-нибудь посетить этот чудесный уголок России. Всего хорошего и до скорой встречи!',
 'Здравствуй, моя милая Анютка! Мы наконец-то добрались до Сочи, города-курорта на берегу Черного моря. Здесь так красиво и уютно, что хочется остаться навсегда. Вокруг нас высокие горы, покрытые зеленью, а воздух пропитан ароматом цветов и хвои. Сегодня мы прогуляемся по набережной и посмотрим, как живут местные жители. Уже мечтаю о том, как буду рассказывать тебе обо всем увиденном. Береги себя и жди моего возвращения.',
 'Дорогие мои родители! Пишу вам из Москвы, столицы нашей Родины. Город поражает своим величием и разнообразием архитектурных стилей. Сегодня мы посетили Красную площадь и Кремль, откуда открывается потрясающий вид на Москву-реку. Вечером планирую сходить в театр, чтобы насладиться культурной жизнью этого великого города. Надеюсь, что скоро смогу вернуться домой и поделиться с вами всеми впечатлениями. Будьте здоровы и счастливы!',
 'Милый Ванечка! Здравствуйте из солнечной Одессы! Это город контрастов, где старинная архитектура соседствует с современными зданиями, а шумные улицы сменяются тихими парками. Сегодня мы гуляли по Дерибасовской улице и наслаждались атмосферой южного города. Завтра собираемся поехать на пляж, чтобы позагорать и искупаться в Черном море. Не забудьте передать привет бабушке и дедушке. До скорого!',
 'Здравствуйте, дорогие друзья! Вот уже несколько дней мы находимся в Пятигорске, известном курорте на Северном Кавказе. Здесь можно найти множество минеральных источников, которые помогают восстановить здоровье и силы. Сегодня мы поднялись на гору Машук, откуда открывается великолепный вид на окрестности. Завтра планируем посетить местный музей, чтобы узнать больше об истории этого замечательного места. Желаю вам здоровья и счастья!',
 'Приветствую вас, родные, из Казани! Этот древний город впечатляет своей историей и культурой. Сегодня мы посетили Казанский кремль и мечеть Кул-Шариф, которая является одной из самых больших в Европе. Вечером планируем прогуляться по набережной реки Волги и полюбоваться закатом. Очень надеюсь, что у меня будет возможность приехать сюда еще раз и показать вам все эти красоты. С любовью и теплом.',
 'Добрый день, моя дорогая сестра! Пишу тебе из Ярославля, одного из старейших городов России. Здесь много исторических памятников и музеев, которые рассказывают о богатой истории нашего народа. Сегодня мы посетили Спасо-Преображенский монастырь и насладились его архитектурой. Завтра планируем прогуляться по набережной Волги и попробовать местные блюда. Передавай привет всем нашим родным и друзьям. Жду встречи с тобой!',
 'Привет, мама и папа! Мы прибыли в Нижний Новгород, один из крупнейших городов России. Здесь мы увидели знаменитый Нижегородский кремль и прокатились на канатной дороге над Волгой. Завтра планируем посетить ярмарку, чтобы купить сувениры и попробовать местные деликатесы. Очень хочу, чтобы вы смогли увидеть все это своими глазами. До скорой встречи!',
 'Доброго времени суток, мои любимые! Вот уже неделю мы находимся в Екатеринбурге, городе с богатой историей и культурным наследием. Сегодня мы посетили Храм-на-Крови и музей «Россия — Моя история». Завтра планируем отправиться на экскурсию по местным заводам и фабрикам, чтобы узнать больше о промышленном прошлом Урала. Очень скучаю по дому и жду момента, когда смогу обнять вас всех. Берегите себя!',
 'Дорогие мои, здравствуйте из прекрасного Санкт-Петербурга! Этот город просто завораживает своей красотой и величием. Сегодня мы посетили Эрмитаж и Петропавловскую крепость, откуда открываются потрясающие виды на Неву. Завтра планируем прогуляться по набережным и посмотреть развод мостов. Надеюсь, что однажды и вы сможете приехать сюда и насладиться этим удивительным городом. Люблю вас!']

texts_before_revolution_gigachat = ['Привет тебе из Константинополя! Здесь просто восхитительно! Города будто бы разделен на две части - старую и новую, обе они так разные и одновременно такие прекрасные. Я уже успел посетить множество достопримечательностей, включая Айя-Софию и дворец Топкапы. Кажется, каждый камень здесь хранит свою историю. Здесь также можно насладиться великолепными видами на Босфор и Мраморное море. Уезжаю отсюда с легкой грустью, зная, что такие моменты бывают редко. Как твои дела дома? Пиши мне чаще! Твой друг.',
 'Здравствуй, моя дорогая! Пишу тебе из Крыма, где отдыхаю и набираюсь сил. Море здесь такое теплое и чистое, что кажется, что оно манит тебя к себе снова и снова. Уже успел побывать в Ялте и Севастополе, увидеть знаменитые дворцы и музеи. Особенно впечатлил меня Ласточкино гнездо - сказочное место, словно сошедшее с картинок. Хочу поделиться с тобой этой красотой, ведь такие моменты остаются в памяти навсегда. До скорой встречи, моя любимая!',
 'Привет тебе из Москвы! Столица России встречает нас своими яркими огнями и богатой историей. Мы уже успели посетить Красную площадь, Кремль и Третьяковскую галерею. Ощущение, что каждый уголок этого города дышит прошлым и настоящим. Наслаждаемся каждым моментом, ведь город такой живой и динамичный. Вернемся домой с новыми впечатлениями и воспоминаниями. До скорой встречи, моя дорогая! Твой любящий муж.', '', '', '', '', '', '', '']

texts_before_revolution_gigachat_2 = ['Приветствую вас из Крыма! Приехал сюда недавно и поражен красотой здешних мест. Берег моря такой ласковый, а воздух чистый и свежий. Желаю вам всего самого светлого и радостного. До свидания!',
 'Кисловодск встретил меня приятной прохладой после жаркого переезда. Здесь приятно прогуливаться вечерами, любуясь живописными видами гор. Буду писать еще, до скорого!',
 'Извещаю вас, дорогие мои, что достиг Казани. Город встречает гостеприимно, а местные жители такие радушные. Осмотрюсь немного здесь и отправляюсь дальше. С любовью и теплом.',
 'Баку – истинное чудо света! Каждый уголок здесь дышит историей и загадками Востока. Увидеть его собственными глазами – счастье невероятное. До встречи!',
 'Москва... Родной мой город! Как прекрасно вновь вернуться домой. Столько приятных встреч и знакомств произошло за эти дни. Пусть ваше сердце всегда будет наполнено радостью и счастьем.',
 'Посылаю приветы вам из Рязани. Этот старинный русский город полон тихой прелести и очарования. Обожаю бродить по его узким улочкам и разглядывать старинные здания.',
 'Дорогие сердечки, шлю вам привет из Константинополя. Город незабываемый и волшебный, хочется задержаться тут навсегда. Напишу снова, целую крепко.',
 'Саратов – величавый и красивый! Здешние просторы радуют глаз. Надеюсь увидеться с вами как можно скорее. До новых встреч!',
 'Рязанская губерния встретила нас гостеприимством и уютом. Каждая деревенька кажется чудесным уголком природы. Пусть каждый день приносит радость и покой.',
 'Написала вам письмо с Байкала – красивейшего озера России. Вода здесь чистая и прозрачная, словно стекло. Время пролетело незаметно, надеюсь скоро опять увидеть вас.']


texts_before_revolution_gigachat_2_pro = ['Здравствуй дорогая моя тетушка!\nСлучайно сегодня оказался в славном городе Казань.\nГород действительно красивый, даже завораживает своей восточной атмосферой.\nБольшой Кремль впечатляет своим величием и размахом!\nТепло тебе и здоровья твоему сердцу!\nДо скорой встречи,\nТвой племянник',
 'Прекрасная Владимирская земля встречает нас щедро и радушно, любезная мамуля!\nДивной красоты церкви здесь будто парят над землей своими золотыми куполами, сердце радуется!\nСкоро отправимся смотреть древний Суздаль!\nКрепко тебя обнимаю!',
 'Милая Олечка-прелесть!\nСегодня, благодаря доброте сердца проводника Бори Курочкина, впервые ступил ногами благородными на Тверскую землю!\nЗдесь дух героев дышит в каждом камне и листике кустика вишневого!\nБудьте здоровы, моя дорогая милая сестра!',
 'Привет тебе сердечный, батюшка родной!\nПишу тебе из прелестного города Нижний Новгород!\nВот как увидел Волгу-матушку широкую - сердце затрепетало в груди! \nЖелаю счастья великого тебе!\nЦелую горячо!\nСын твой Левка',
 'Родная моя Настенька!\nТолько-только прибыли в древнюю богатую Вологду! \nВся она кружевами и деревянным узором покрошена — глаз души не оторвешь!\nНа улице такую теплынь развели, солнышко греет ласково... \nСкоро домой поспешим!',
 'Добрый день всему нашему семейству дорогому!\nГостила нынче день чудесный в Нижнем Новгороде \n— кто хоть раз побывал тут, тот уже навек пленен будет красотою здешних мест!\nОбещаю вас скоро повидать!\nВсегда ваша Татьяна',
 'Сердечно привету тебя мой добрый друг Михаил Иванович!\nВчера утром прибыл в живописную Воронежскую губернию.\nГлубоко вздохнул полной грудью чистого воздуха вольной природы русской! \nЖелаем здравия вам и вашим родным!',
 'Ах, милый Петенька-племянничек дорогой мой!\nБыли в великолепном Екатеринбурге!\nНе город, а целая галерея старинных особняков прекрасных стоит себе непринужденно посреди живописных холмов!\nГорячей любви моей тебе хочу скорее передать!',
 'Здравия желаю всей нашей родне любимой!\nОстановились на короткое время в Ростове-на-Дону!\nТут такого изобилия фруктов всяких-сочных видим, каких прежде не видывали ни разу!\nНизкий поклон и горячий поцелуй вашим душам от меня нежных!',
 'Многоуважаемая Прасковья Павловна!\nОтдыхаем прекрасно в Калуге проулочками познавательными гуляя среди музеев научных разнообразных! \nВаш постоянный почитатель Григорий']

texts_before_revolution_gigachat_2_max = ['Милый друг мой! Из Сочи шлешь тебе поклон горячий. Здесь море шумит ласково, воздух чист и свеж, пальмы склоняют ветви, словно хотят обнять путника. Я гулял сегодня вдоль берега, слушал пение птиц и шелест волн. Желаю тебе счастья, здоровья крепкого и душевного покоя. До скорого свидания!',
 'Дорогой Сереженька! Приветствую тебя из Нижнего Новгорода. Город просторный, величественный, стоит на высоком берегу Волги. Прогулялся вчера вечером по набережной — дух захватило от красоты вечернего города. Счастья тебе, радости и успехов во всём! Твой любящий дядя.',
 'Здравствуй, дорогая сестрица моя! Пришла тебе весточка из Самары. Удивительное место — здесь так красиво и спокойно. Город раскинулся широко, дома стоят уютно, улицы тихие. Пусть счастье будет твоим спутником всегда! Целовать тебя целую!',
 'Приветствую вас, мои дорогие, из Одессы! Прекрасное южное солнце греет мою душу, волны Чёрного моря нежно плещут под окнами гостиницы. Природа великолепна, люди доброжелательны. Буду скучать без вас! Всех благ вам, друзья мои сердечные!',
 'Шлю поклоны моим родным и любимым из Ростова-на-Дону. Остановился в этом городе ненадолго, полюбовался Доном широкой рекой, насладился вкусной донской рыбой. Пусть радость ваша будет полной чашей, здоровье крепким как Дон-батюшка! Обнимаю крепко-нежно!',
 'Моя милая подруженька, шлю тебе приветы горячие из Харькова. Город студенческий, веселый, оживленный. Людей много, жизнь кипит ключом. На площади Университетской побывала - деревья там высокие-превысокие, зелёные-презелёные. Любви тебе истинной, добра искреннего желаю!',
 'Мой дорогой товарищ, отправляю тебе приветствие из Астрахани. Вот уж город удивительный - рыбакам рай земной, Волгой окружен со всех сторон. Пробовал рыбу горячего копчения, аппетит разбудилась моментально. Пусть твои дни будут такими же светлыми и радостными, как здешнее небо ясное!',
 'Солнечный Санкт-Петербург передаёт тебе поклон, любимый племянник. Великолепная архитектура Петербурга восхищает взор каждого приезжего гостя. Невский проспект полон народу, Адмиралтейство сияет золотом куполов. Благополучия тебе и удачи во всех начинаниях моих милых родных людей!',
 'Дражайшая Катюшечка, присылаю приветик тебе из Саратова. Завтра утром отплывём вниз по Волге дальше путешествовать. Живописнейшие виды открылись мне здесь - белые пароходы плавают плавно, птицы летают над головой. Будьте здоровы, родные мои сердцу близкие люди!',
 'Сердце моё дорогое семейство, вот и Добринка решила послать вам пару словечек своих теплых из Ярославля. Замечательный город старинный, Кремль красив необычайно, река Волга широкая-прекрасная течёт рядом. Отправляю свою любовь бескрайнюю и пожелания благополучия вам!']

df = pd.DataFrame(texts_before_revolution_gigachat, index=[i for i in range(10)], columns=["GigaChat"])
df["GigaChat2"] = texts_before_revolution_gigachat_2
df["GigaChat-Max"] = texts_before_revolution_gigachat_max
df["GigaChat-Pro"] = texts_before_revolution_gigachat_pro
df["GigaChat-2-Max"] = texts_before_revolution_gigachat_2_max
df["GigaChat-2-Pro"] = texts_before_revolution_gigachat_2_pro
df

,GigaChat,GigaChat2,GigaChat-Max,GigaChat-Pro,GigaChat-2-Max,GigaChat-2-Pro
0,Привет тебе из Константинополя! Здесь просто в...,Приветствую вас из Крыма! Приехал сюда недавно...,"Приветствую тебя, мой дорогой друг, из живопис...","Приветствую тебя из Казани, милая моя сестрица...",Милый друг мой! Из Сочи шлешь тебе поклон горя...,Здравствуй дорогая моя тетушка!\nСлучайно сего...
1,"Здравствуй, моя дорогая! Пишу тебе из Крыма, г...",Кисловодск встретил меня приятной прохладой по...,"Здравствуй, моя милая Анютка! Мы наконец-то до...","Дорогая Софья, шлю тебе привет из Нижнего Новг...",Дорогой Сереженька! Приветствую тебя из Нижнег...,Прекрасная Владимирская земля встречает нас ще...
2,Привет тебе из Москвы! Столица России встречае...,"Извещаю вас, дорогие мои, что достиг Казани. Г...","Дорогие мои родители! Пишу вам из Москвы, стол...","Здравствуй, дорогой брат Владимир! Вот я и в П...","Здравствуй, дорогая сестрица моя! Пришла тебе ...","Милая Олечка-прелесть!\nСегодня, благодаря доб..."
3,,Баку – истинное чудо света! Каждый уголок здес...,Милый Ванечка! Здравствуйте из солнечной Одесс...,"Приветствую тебя из Саратова, любимая моя жена...","Приветствую вас, мои дорогие, из Одессы! Прекр...","Привет тебе сердечный, батюшка родной!\nПишу т..."
4,,Москва... Родной мой город! Как прекрасно внов...,"Здравствуйте, дорогие друзья! Вот уже нескольк...","Дорогая Мария, здравствуй! Я в Екатеринбурге, ...",Шлю поклоны моим родным и любимым из Ростова-н...,Родная моя Настенька!\nТолько-только прибыли в...
5,,Посылаю приветы вам из Рязани. Этот старинный ...,"Приветствую вас, родные, из Казани! Этот древн...","Приветствую тебя из Иркутска, мой верный друг!...","Моя милая подруженька, шлю тебе приветы горячи...",Добрый день всему нашему семейству дорогому!\n...
6,,"Дорогие сердечки, шлю вам привет из Константин...","Добрый день, моя дорогая сестра! Пишу тебе из ...","Здравствуй, дорогая Ольга! Вот я и в Архангель...","Мой дорогой товарищ, отправляю тебе приветстви...",Сердечно привету тебя мой добрый друг Михаил И...
7,,Саратов – величавый и красивый! Здешние просто...,"Привет, мама и папа! Мы прибыли в Нижний Новго...","Приветствую тебя из Красноярска, моя любимая д...",Солнечный Санкт-Петербург передаёт тебе поклон...,"Ах, милый Петенька-племянничек дорогой мой!\nБ..."
8,,Рязанская губерния встретила нас гостеприимств...,"Доброго времени суток, мои любимые! Вот уже не...","Дорогая Лиза, здравствуй! Я в Чите, городе на ...","Дражайшая Катюшечка, присылаю приветик тебе из...",Здравия желаю всей нашей родне любимой!\nОстан...
9,,Написала вам письмо с Байкала – красивейшего о...,"Дорогие мои, здравствуйте из прекрасного Санкт...","Приветствую тебя из Владивостока, мой верный т...","Сердце моё дорогое семейство, вот и Добринка р...",Многоуважаемая Прасковья Павловна!\nОтдыхаем п...


Вывод: благодаря детальному промпт-инжинирингу (прописыванию особенностей дореволюционного стиля написания открыток, например, уменьшительно-ласкательные обращения, метафоричность описания, специфичные для того времени города для путешествия, сокращения, последовательность изложения, в системном промпте; а также few-shot подхода к промптингу, использовании примеров открыток из корпуса "Пишу тебе" в качестве примера стилистического и лингвистического оформления открыток), а также настройки параметров модели (температура модели в диапазоне от 1 до 1.2) получилось сгенерировать стилистически похожие примеры открыток.

Наиболее удачным (на мой взгляд) являются примеры модели GigaChat-2-Pro. Самим приближенными к оригинальным открыткам являются следующие особенности: формат обращения к адресату, как и более частотные имена для того времени, так и характерные уменьшительно-ласкательные формы обращения ("Многоуважаемая Прасковья Павловна", "дорогая моя Настенька", "Ах, милый Петенька-племянничек дорогой мой!"), художественность в описании путешествия и места ("Вся она кружевами и деревянным узором покрошена — глаз души не оторвешь!", "...чистого воздуха вольной природы русской!", "кто хоть раз побывал тут, тот уже навек пленен будет красотою здешних мест!"), специфичная лексика ("Прекрасная Владимирская земля", "Живописная Воронежская губерния", "Гостила нынче день чудесный"), а также использование популярных направлений для путешествий в дероволюционное время (Суздаль, Владимир, Воронежская губерния). В отличие от результатов других моделей GigaChat-2-Pro более приближено к оригиналу сгенерировал примеры открыток

Минусы prompt-подхода для генерации открыток: если в случае со стилистикой и лингвистическими особенностями написания открыток при выборе определенной модели и должном промпт-подходе и экспериментах можно добиться результата, приближенного к оригиналу, то что касается фактологической точности (исключение современных городов, часто в сгенерированных текстах прослеживался Калининград (именно в этом написании), хотя он был основан только в 1945г., а также использования современных городов-миллионников, например, Екатеринбурга и Красноярска, которые являются популярными туристическими направлениями сегодня, а не в 19 веке), а также некоторых особенностей оформления (использование сокращений), то данные аспекты плохо генерируются LLM.
В связи с этим, для минимизации данных галлюцинаций необходимо использовать тонкую настройку LLM (LORA, QLORA fune-tinung на датасете "Пишу тебе") и проводить генерации с данной надстройкой

Также необходимо создать на основе данных корпуса классификатор (классический ML, ансамблевые методы, перценптроны, трансформеры) для классификации открыток согласно году написания, который поможет (при высокой f1-мере) количественно оценить результаты генерации открыток с помощью LLM (как и при prompt-подходе, так и при fune-tuning)